In [1]:
import cv2
import numpy as np
import os
import sys

from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier


IMG_WIDTH = 300 
IMG_HEIGHT = 300

In [2]:
data_dir = './coins_dataset_combined' #data directory
IMG_WIDTH = 300 
IMG_HEIGHT = 300
NUM_CATEGORIES = 3
TEST_SIZE = 0.25
mapping_label = {'10c':0,
  '20c':1,
  '5c':2}

def load_data(dir):
    # initialise list of images
    images = []
    # initialise list of labels
    labels = []
    for root, dirs, files in os.walk(dir):
        
        for name in files:
            path = os.path.join(root, name)
            img = Image.open(path)
            img=img.rotate(4)
            area = (400, 250, 950, 800)
            img = img.crop(area)
            
            pil_image = img.convert('RGB') 
            open_cv_image = np.array(pil_image) 
            # Convert RGB to BGR 
            resized_image = open_cv_image[:, :, ::-1].copy() 
            # Convert to grayscale.
            gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

            # Blur using 3 * 3 kernel.
            gray_blurred = cv2.blur(gray, (3, 3))

            # Apply Hough transform on the blurred image.
            detected_circles = cv2.HoughCircles(gray_blurred, 
                               cv2.HOUGH_GRADIENT, 1, 50, param1 = 100,
                           param2 = 100, minRadius = 100, maxRadius = 300)

            # Draw circles that are detected.
            if detected_circles is not None:

                # Convert the circle parameters a, b and r to integers.
                detected_circles = np.uint16(np.around(detected_circles))

                for pt in detected_circles[0, :]:
                    a, b, r = pt[0], pt[1], pt[2]
                    print(a,b,r)
                    break #adds first(biggest circle) radius only
            
            # add resized image to images list
            images.append(r)
            # add category to labels array
            path=os.path.dirname(path)
            labels.append(mapping_label[os.path.basename(path)])
    return images,labels

images, labels = load_data(data_dir)
print(images)
print(labels)

labels = tf.keras.utils.to_categorical(labels)
print(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE, random_state=3237
)
print(x_train.shape,y_train.shape, x_test.shape,y_test.shape)


282 226 161
276 346 165
262 270 167
260 286 166
314 226 177
316 342 168
268 290 165
266 348 166
258 212 164
242 266 162
258 334 178
248 348 167
230 126 163
212 152 150
280 208 166
266 252 165
284 164 176
276 258 162
256 232 162
250 344 172
290 236 178
270 346 166
290 198 164
276 262 165
282 186 166
272 290 166
220 148 168
212 232 166
212 298 166
338 210 167
332 318 169
244 338 164
254 350 165
282 200 166
292 346 167
294 82 170
278 240 161
304 204 165
292 236 167
350 222 166
340 256 167
276 254 167
268 322 160
288 168 160
270 196 168
306 224 157
290 302 169
286 264 166
290 348 167
282 224 178
276 346 166
196 342 165
200 340 167
196 338 161
192 340 165
336 268 170
348 366 175
234 224 168
230 306 170
318 338 171
322 362 171
222 298 169
222 354 168
202 232 164
200 282 170
176 312 168
168 344 171
228 262 172
242 342 179
336 132 156
342 214 178
260 156 166
264 252 167
228 344 166
240 350 167
332 172 152
340 230 171
238 354 174
248 346 172
286 222 167
278 224 163
230 286 170
254 290 169
270 3

In [3]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

knn.fit(x_train.reshape(-1, 1), y_train)
print(knn.score(x_test.reshape(-1, 1), y_test))

0.948051948051948


In [ ]:
import joblib

joblib.dump(knn, 'knn_hough.pkl') 